In [15]:
from datasets import load_dataset
from torch.utils.data.sampler import BatchSampler, RandomSampler
from torch.utils.data import DataLoader
import torchaudio
from transformers import pipeline

from torchaudio.transforms import Resample
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F

In [3]:
# Загружаем датасет
cv_11 = load_dataset("mozilla-foundation/common_voice_11_0", "ru", split="train")

/home/redalexdad/anaconda3/envs/dl_science/lib/python3.9/site-packages/datasets/load.py:1454: FutureWarning: The repository for mozilla-foundation/common_voice_11_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_11_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [4]:
# Создаем батч-сэмплер и загружаем данные
batch_sampler = BatchSampler(RandomSampler(cv_11), batch_size=32, drop_last=False)
dataloader = DataLoader(cv_11, batch_sampler=batch_sampler)

In [5]:
# Подготавливаем датасет
ds = load_dataset("mozilla-foundation/common_voice_11_0", "ru", use_auth_token=True)

/home/redalexdad/anaconda3/envs/dl_science/lib/python3.9/site-packages/datasets/load.py:2508: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(
Using the latest cached version of the module from /home/redalexdad/.cache/huggingface/modules/datasets_modules/datasets/mozilla-foundation--common_voice_11_0/3f27acf10f303eac5b6fbbbe02495aeddb46ecffdb0a2fe3507fcfbf89094631 (last modified on Mon Feb 12 09:07:13 2024) since it couldn't be found locally at mozilla-foundation/common_voice_11_0, or remotely on the Hugging Face Hub.


In [6]:
def prepare_dataset(batch):
    transcription = batch["sentence"]
  
    if transcription.startswith('"') and transcription.endswith('"'):
        transcription = transcription[1:-1]
  
    if transcription[-1] not in [".", "?", "!"]:
        transcription = transcription + "."
  
    batch["sentence"] = transcription
  
    return batch

In [7]:
ds = ds.map(prepare_dataset, desc="preprocess dataset")

preprocess dataset:   0%|          | 0/22862 [00:00<?, ? examples/s]

preprocess dataset:   0%|          | 0/9629 [00:00<?, ? examples/s]

preprocess dataset:   0%|          | 0/9630 [00:00<?, ? examples/s]

preprocess dataset:   0%|          | 0/16865 [00:00<?, ? examples/s]

preprocess dataset:   0%|          | 0/7159 [00:00<?, ? examples/s]

In [8]:
# Загружаем предварительно обученную модель для распознавания речи
asr = pipeline("automatic-speech-recognition", model="facebook/wav2vec2-base-960h")

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You sho

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

In [21]:
# Задаем целевую длину аудиозаписей
target_length = 160000

# Создаем объект Resample для приведения к целевой длине
resampler = Resample(orig_freq=16000, new_freq=16000)

# Проходим по первым 5 аудиозаписям и распознаем речь
for i, batch in enumerate(dataloader):
    if i >= 5:
        break
    
    audio_data, _ = torchaudio.load(batch["path"])
    
    # Применяем ресемплирование к целевой длине
    audio_data = resampler(audio_data)

    # Выполняем динамическое заполнение или обрезаем до целевой длины
    if audio_data.size(1) < target_length:
        padded_audio_data = F.pad(audio_data, (0, target_length - audio_data.size(1)))
    else:
        # Обрезаем лишние кадры
        audio_data = audio_data[:, :target_length]

    transcriptions = asr(audio_data.numpy())

    for i, transcription in enumerate(transcriptions):
        print(f"Аудиозапись: {batch['path'][i]}")
        print(f"Текст: {transcription['sentence']}")
        print("="*50)

RuntimeError: stack expects each tensor to be equal size, but got [184896] at entry 0 and [205056] at entry 1